<B> Importing Required Libraries.....

In [52]:
import random # library for random number generation
import requests
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs


import json
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
import matplotlib.colors as colors
import matplotlib as mpl
mpl.style.use('ggplot')

print('Libraries imported.')

WARNING conda.base.context:use_only_tar_bz2(632): Conda is constrained to only using the old .tar.bz2 file format because you have conda-build installed, and it is <3.18.3.  Update or remove conda-build to get smaller downloads and faster extractions.
Solving environment: failed

PackagesNotFoundError: The following packages are not available from current channels:

  - anaconda/linux-64::grpcio==1.16.1=py36hf8bcb03_1 -> openssl[version='>=1.1.1,<1.1.2.0a0']

Current channels:

  - https://conda.anaconda.org/conda-forge/linux-64
  - https://conda.anaconda.org/conda-forge/noarch
  - https://repo.anaconda.com/pkgs/main/linux-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/linux-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.


WARNING conda.base.context:use_only_tar_bz2(632): Co

<B> Getting the Chicago's Socio Economic Data from a CSV file

In [3]:
filename = 'Chicago_SocioEconomicData.csv'
df_ChicSocioEco = pd.read_csv(filename)
df_ChicSocioEco.head()

,Community Area Number,COMMUNITY AREA NAME,PERCENT HOUSEHOLDS ABOVE POVERTY,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 25+ WITH HIGH SCHOOL DIPLOMA,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED BETWEEN AGE 18 AND 64,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME
0,1.0,Rogers Park,76.4,23.6,81.8,18.2,72.5,27.5,23939
1,2.0,West Ridge,82.8,17.2,79.2,20.8,61.5,38.5,23040
2,3.0,Uptown,76.0,24.0,88.2,11.8,77.8,22.2,35787
3,4.0,Lincoln Square,89.1,10.9,86.6,13.4,74.5,25.5,37524
4,5.0,North Center,92.5,7.5,95.5,4.5,73.8,26.2,57123


<B> Data cleaning and Preprocessing

In [4]:
df_final = df_ChicSocioEco.drop('PERCENT HOUSEHOLDS BELOW POVERTY', axis=1)
df_final = df_final.drop('PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA', axis=1)
df_final = df_final.drop('PERCENT AGED UNDER 18 OR OVER 64', axis=1)


df_final.head()

,Community Area Number,COMMUNITY AREA NAME,PERCENT HOUSEHOLDS ABOVE POVERTY,PERCENT AGED 25+ WITH HIGH SCHOOL DIPLOMA,PERCENT AGED BETWEEN AGE 18 AND 64,PER CAPITA INCOME
0,1.0,Rogers Park,76.4,81.8,72.5,23939
1,2.0,West Ridge,82.8,79.2,61.5,23040
2,3.0,Uptown,76.0,88.2,77.8,35787
3,4.0,Lincoln Square,89.1,86.6,74.5,37524
4,5.0,North Center,92.5,95.5,73.8,57123


<B> Getting the top community area based on following criteria: Highest per capita income, high % of household above poverty, % of population with age between 18 and 64 and % aged 25+ with high school diploma at least. 

In [5]:
df_final = df_final.sort_values(['PER CAPITA INCOME ', 'PERCENT AGED BETWEEN AGE 18 AND 64', 'PERCENT HOUSEHOLDS ABOVE POVERTY', 'PERCENT AGED 25+ WITH HIGH SCHOOL DIPLOMA'], ascending=[False, False, False, False])
df_final.head()
#list(df_final.columns.values)

,Community Area Number,COMMUNITY AREA NAME,PERCENT HOUSEHOLDS ABOVE POVERTY,PERCENT AGED 25+ WITH HIGH SCHOOL DIPLOMA,PERCENT AGED BETWEEN AGE 18 AND 64,PER CAPITA INCOME
7,8.0,Near North Side,87.1,97.5,77.4,88669
6,7.0,Lincoln Park,87.7,96.4,78.5,71551
31,32.0,Loop,85.3,96.9,86.5,65526
5,6.0,Lake View,88.6,97.4,83.0,60058
32,33.0,Near South Side,86.2,92.6,78.2,59077


<B> Get the top Community from the dataframe and the Latitude & Longitude for it.

In [6]:
top_community = df_final.loc[df_final.index[0], 'COMMUNITY AREA NAME']
top_community = top_community + ', IL'
top_community

'Near North Side, IL'

In [7]:
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(top_community)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

41.9000327 -87.6344975


In [8]:
CLIENT_ID = 'YPEJXGGFUM3FBCZP4O4ZDJYYARC0Q1TUGO5XWQLEXFYWKU1G' # your Foursquare ID
CLIENT_SECRET = 'JUUMMRQIBZWT1WXVOOJMH03TBXCR4KMJ02T2XP2O15IWHZXI' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
radius = 1000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url
results = requests.get(url).json()


<B> Get Relevant part of the JSON file

In [9]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4aa01fd8f964a5206a3e20e3',
  'name': "Kiki's Bistro",
  'location': {'address': '900 N Franklin St',
   'crossStreet': 'at W Locust St',
   'lat': 41.89894713032016,
   'lng': -87.63589538908546,
   'labeledLatLngs': [{'label': 'display',
     'lat': 41.89894713032016,
     'lng': -87.63589538908546}],
   'distance': 167,
   'postalCode': '60610',
   'cc': 'US',
   'city': 'Chicago',
   'state': 'IL',
   'country': 'United States',
   'formattedAddress': ['900 N Franklin St (at W Locust St)',
    'Chicago, IL 60610',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d10c941735',
    'name': 'French Restaurant',
    'pluralName': 'French Restaurants',
    'shortName': 'French',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/french_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'c

In [10]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [11]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng', 'venue.id']
#filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(5)

,name,categories,lat,lng,id
0,Kiki's Bistro,French Restaurant,41.898947,-87.635895,4aa01fd8f964a5206a3e20e3
1,BIG & little's Restaurant,Fast Food Restaurant,41.898356,-87.637094,4aef2171f964a520ded521e3
2,Hi-Fi Personal Fitness,Gym,41.897371,-87.637342,4b9e1e3cf964a520accb36e3
3,Headquarters Beercade,Bar,41.897221,-87.634879,52f139a1498e481bc2d24e0d
4,Fitness Formula Club (FFC),Gym / Fitness Center,41.901407,-87.631743,4b257751f964a5205d7224e3


In [13]:
print('{} venues were returned by Foursquare.'.format(dataframe_filtered.shape[0]))

100 venues were returned by Foursquare.


In [29]:
print('There are {} uniques categories.'.format(len(dataframe_filtered['categories'].unique())))

There are 55 uniques categories.


In [14]:
dataframe_filtered['categories'].unique()

array(['French Restaurant', 'Fast Food Restaurant', 'Gym', 'Bar',
       'Gym / Fitness Center', 'Paper / Office Supplies Store',
       'American Restaurant', 'Pizza Place', 'Beer Bar', 'Cycle Studio',
       'Dance Studio', 'Yoga Studio', 'Hotel', 'Restaurant', 'Spa',
       'Steakhouse', 'BBQ Joint', 'New American Restaurant',
       'Massage Studio', 'Lounge', 'Dessert Shop', 'Cocktail Bar',
       'Grocery Store', 'Boutique', 'Optical Shop',
       'Vietnamese Restaurant', 'Hotel Bar', 'Breakfast Spot',
       'Coffee Shop', 'Italian Restaurant', "Men's Store",
       'Seafood Restaurant', 'Pub', "Women's Store", 'Diner',
       'Department Store', 'Farmers Market', 'Burger Joint',
       'Korean Restaurant', 'Clothing Store', 'Latin American Restaurant',
       'Tapas Restaurant', 'Café', 'Health Food Store', 'Juice Bar',
       'Cupcake Shop', 'Bakery', 'Mexican Restaurant',
       'Furniture / Home Store', 'Salad Place', 'Sushi Restaurant',
       'Speakeasy', 'Brazilian Restau

In [27]:
# one hot encoding
Chicago_onehot = pd.get_dummies(dataframe_filtered['categories'], prefix="", prefix_sep="")
# add Latitude and Longitude column back to dataframe
#Chicago_onehot['lat'] = dataframe_filtered['lat'] 

# move lat column to the first column
#fixed_columns = [Chicago_onehot.columns[-1]] + list(Chicago_onehot.columns[:-1])
#Chicago_onehot = Chicago_onehot[fixed_columns]

#Chicago_onehot['lng'] = dataframe_filtered['lng'] 

# move lng column to the first column
#fixed_columns = [Chicago_onehot.columns[-1]] + list(Chicago_onehot.columns[:-1])
#Chicago_onehot = Chicago_onehot[fixed_columns]


Chicago_onehot['id'] = dataframe_filtered['id'] 

# move lng column to the first column
fixed_columns = [Chicago_onehot.columns[-1]] + list(Chicago_onehot.columns[:-1])
Chicago_onehot = Chicago_onehot[fixed_columns]

Chicago_onehot.head()

,id,American Restaurant,BBQ Joint,Bakery,Bar,Beer Bar,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,...,Seafood Restaurant,Shoe Store,Spa,Speakeasy,Steakhouse,Sushi Restaurant,Tapas Restaurant,Vietnamese Restaurant,Women's Store,Yoga Studio
0,4aa01fd8f964a5206a3e20e3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4aef2171f964a520ded521e3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4b9e1e3cf964a520accb36e3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,52f139a1498e481bc2d24e0d,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4b257751f964a5205d7224e3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
Chicago_onehot.shape

(100, 57)

In [63]:
chicago_grouped = Chicago_onehot.groupby(['id']).mean().reset_index()
chicago_grouped

,id,American Restaurant,BBQ Joint,Bakery,Bar,Beer Bar,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,...,Seafood Restaurant,Shoe Store,Spa,Speakeasy,Steakhouse,Sushi Restaurant,Tapas Restaurant,Vietnamese Restaurant,Women's Store,Yoga Studio
0,40b28c80f964a520e4fb1ee3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,40db6b00f964a52078011fe3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,4176fc00f964a520b51d1fe3,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,44198f48f964a52029311fe3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,4a176a52f964a52058791fe3,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,4a35674cf964a520f49c1fe3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,4a444bd6f964a5206ea71fe3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,4a5bcc8ef964a520d1bb1fe3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,4a71e715f964a520d3d91fe3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,4a736375f964a52054dc1fe3,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


<B> Run *k*-means to cluster the neighborhood into 5 clusters.

In [64]:
# set number of clusters
kclusters = 10

#chicago_grouped_clustering = chicago_grouped.drop('lat', 1)
#chicago_grouped_clustering = chicago_grouped_clustering.drop('lng', 1)
chicago_grouped_clustering = chicago_grouped.drop('id', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(chicago_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 3, 2, 0, 0, 0, 0, 0], dtype=int32)

<b> Run *k*-means to cluster the neighborhood into 5 clusters.

In [65]:
# add clustering labels
chicago_grouped.insert(0, 'Cluster Labels', kmeans.labels_)
chicago_grouped

,Cluster Labels,id,American Restaurant,BBQ Joint,Bakery,Bar,Beer Bar,Bookstore,Boutique,Brazilian Restaurant,...,Seafood Restaurant,Shoe Store,Spa,Speakeasy,Steakhouse,Sushi Restaurant,Tapas Restaurant,Vietnamese Restaurant,Women's Store,Yoga Studio
0,0,40b28c80f964a520e4fb1ee3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,40db6b00f964a52078011fe3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,2,4176fc00f964a520b51d1fe3,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,44198f48f964a52029311fe3,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,2,4a176a52f964a52058791fe3,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,4a35674cf964a520f49c1fe3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,4a444bd6f964a5206ea71fe3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,4a5bcc8ef964a520d1bb1fe3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,4a71e715f964a520d3d91fe3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,4a736375f964a52054dc1fe3,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [66]:
chicago_top_location_merged = dataframe_filtered

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
chicago_top_location_merged = chicago_top_location_merged.join(chicago_grouped.set_index('id'), on='id')

chicago_top_location_merged.head()

,name,categories,lat,lng,id,Cluster Labels,American Restaurant,BBQ Joint,Bakery,Bar,...,Seafood Restaurant,Shoe Store,Spa,Speakeasy,Steakhouse,Sushi Restaurant,Tapas Restaurant,Vietnamese Restaurant,Women's Store,Yoga Studio
0,Kiki's Bistro,French Restaurant,41.898947,-87.635895,4aa01fd8f964a5206a3e20e3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,BIG & little's Restaurant,Fast Food Restaurant,41.898356,-87.637094,4aef2171f964a520ded521e3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hi-Fi Personal Fitness,Gym,41.897371,-87.637342,4b9e1e3cf964a520accb36e3,7,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Headquarters Beercade,Bar,41.897221,-87.634879,52f139a1498e481bc2d24e0d,9,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,Fitness Formula Club (FFC),Gym / Fitness Center,41.901407,-87.631743,4b257751f964a5205d7224e3,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<B> First Cluster

In [67]:
chicago_top_location_merged.loc[chicago_top_location_merged['Cluster Labels'] == 0, chicago_top_location_merged.columns[[1] + list(range(5, chicago_top_location_merged.shape[1]))]]

,categories,Cluster Labels,American Restaurant,BBQ Joint,Bakery,Bar,Beer Bar,Bookstore,Boutique,Brazilian Restaurant,...,Seafood Restaurant,Shoe Store,Spa,Speakeasy,Steakhouse,Sushi Restaurant,Tapas Restaurant,Vietnamese Restaurant,Women's Store,Yoga Studio
0,French Restaurant,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Fast Food Restaurant,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Paper / Office Supplies Store,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Beer Bar,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Cycle Studio,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,Dance Studio,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,Yoga Studio,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
15,Spa,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
17,BBQ Joint,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18,New American Restaurant,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [68]:
chicago_top_location_merged.loc[chicago_top_location_merged['Cluster Labels'] == 1, chicago_top_location_merged.columns[[1] + list(range(5, chicago_top_location_merged.shape[1]))]]

,categories,Cluster Labels,American Restaurant,BBQ Joint,Bakery,Bar,Beer Bar,Bookstore,Boutique,Brazilian Restaurant,...,Seafood Restaurant,Shoe Store,Spa,Speakeasy,Steakhouse,Sushi Restaurant,Tapas Restaurant,Vietnamese Restaurant,Women's Store,Yoga Studio
38,Coffee Shop,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49,Coffee Shop,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
68,Coffee Shop,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
79,Coffee Shop,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
86,Coffee Shop,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [69]:
chicago_top_location_merged.loc[chicago_top_location_merged['Cluster Labels'] == 2, chicago_top_location_merged.columns[[1] + list(range(5, chicago_top_location_merged.shape[1]))]]

,categories,Cluster Labels,American Restaurant,BBQ Joint,Bakery,Bar,Beer Bar,Bookstore,Boutique,Brazilian Restaurant,...,Seafood Restaurant,Shoe Store,Spa,Speakeasy,Steakhouse,Sushi Restaurant,Tapas Restaurant,Vietnamese Restaurant,Women's Store,Yoga Studio
6,American Restaurant,2,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51,American Restaurant,2,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
55,American Restaurant,2,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
63,American Restaurant,2,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
69,American Restaurant,2,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
87,American Restaurant,2,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [70]:
chicago_top_location_merged.loc[chicago_top_location_merged['Cluster Labels'] == 3, chicago_top_location_merged.columns[[1] + list(range(5, chicago_top_location_merged.shape[1]))]]

,categories,Cluster Labels,American Restaurant,BBQ Joint,Bakery,Bar,Beer Bar,Bookstore,Boutique,Brazilian Restaurant,...,Seafood Restaurant,Shoe Store,Spa,Speakeasy,Steakhouse,Sushi Restaurant,Tapas Restaurant,Vietnamese Restaurant,Women's Store,Yoga Studio
16,Steakhouse,3,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
22,Steakhouse,3,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
57,Steakhouse,3,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
75,Steakhouse,3,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
82,Steakhouse,3,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [71]:
chicago_top_location_merged.loc[chicago_top_location_merged['Cluster Labels'] == 4, chicago_top_location_merged.columns[[1] + list(range(5, chicago_top_location_merged.shape[1]))]]

,categories,Cluster Labels,American Restaurant,BBQ Joint,Bakery,Bar,Beer Bar,Bookstore,Boutique,Brazilian Restaurant,...,Seafood Restaurant,Shoe Store,Spa,Speakeasy,Steakhouse,Sushi Restaurant,Tapas Restaurant,Vietnamese Restaurant,Women's Store,Yoga Studio
4,Gym / Fitness Center,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,Gym / Fitness Center,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
72,Gym / Fitness Center,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
74,Gym / Fitness Center,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
90,Gym / Fitness Center,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,categories,Cluster Labels,American Restaurant,BBQ Joint,Bakery,Bar,Beer Bar,Bookstore,Boutique,Brazilian Restaurant,...,Seafood Restaurant,Shoe Store,Spa,Speakeasy,Steakhouse,Sushi Restaurant,Tapas Restaurant,Vietnamese Restaurant,Women's Store,Yoga Studio
14,Restaurant,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23,Restaurant,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [78]:
chicago_top_location_merged.loc[chicago_top_location_merged['Cluster Labels'] == 5, chicago_top_location_merged.columns[[1] + list(range(5, chicago_top_location_merged.shape[1]))]]

,categories,Cluster Labels,American Restaurant,BBQ Joint,Bakery,Bar,Beer Bar,Bookstore,Boutique,Brazilian Restaurant,...,Seafood Restaurant,Shoe Store,Spa,Speakeasy,Steakhouse,Sushi Restaurant,Tapas Restaurant,Vietnamese Restaurant,Women's Store,Yoga Studio
14,Restaurant,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23,Restaurant,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
chicago_top_location_merged.loc[chicago_top_location_merged['Cluster Labels'] == 6, chicago_top_location_merged.columns[[1] + list(range(5, chicago_top_location_merged.shape[1]))]]

,categories,Cluster Labels,American Restaurant,BBQ Joint,Bakery,Bar,Beer Bar,Bookstore,Boutique,Brazilian Restaurant,...,Seafood Restaurant,Shoe Store,Spa,Speakeasy,Steakhouse,Sushi Restaurant,Tapas Restaurant,Vietnamese Restaurant,Women's Store,Yoga Studio
7,Pizza Place,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26,Pizza Place,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
56,Pizza Place,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [74]:
chicago_top_location_merged.loc[chicago_top_location_merged['Cluster Labels'] == 7, chicago_top_location_merged.columns[[1] + list(range(5, chicago_top_location_merged.shape[1]))]]

,categories,Cluster Labels,American Restaurant,BBQ Joint,Bakery,Bar,Beer Bar,Bookstore,Boutique,Brazilian Restaurant,...,Seafood Restaurant,Shoe Store,Spa,Speakeasy,Steakhouse,Sushi Restaurant,Tapas Restaurant,Vietnamese Restaurant,Women's Store,Yoga Studio
2,Gym,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29,Gym,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30,Gym,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [76]:
chicago_top_location_merged.loc[chicago_top_location_merged['Cluster Labels'] == 8, chicago_top_location_merged.columns[[1] + list(range(5, chicago_top_location_merged.shape[1]))]]

,categories,Cluster Labels,American Restaurant,BBQ Joint,Bakery,Bar,Beer Bar,Bookstore,Boutique,Brazilian Restaurant,...,Seafood Restaurant,Shoe Store,Spa,Speakeasy,Steakhouse,Sushi Restaurant,Tapas Restaurant,Vietnamese Restaurant,Women's Store,Yoga Studio
13,Hotel,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21,Hotel,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
48,Hotel,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
64,Hotel,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
83,Hotel,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
85,Hotel,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [77]:
chicago_top_location_merged.loc[chicago_top_location_merged['Cluster Labels'] == 9, chicago_top_location_merged.columns[[1] + list(range(5, chicago_top_location_merged.shape[1]))]]

,categories,Cluster Labels,American Restaurant,BBQ Joint,Bakery,Bar,Beer Bar,Bookstore,Boutique,Brazilian Restaurant,...,Seafood Restaurant,Shoe Store,Spa,Speakeasy,Steakhouse,Sushi Restaurant,Tapas Restaurant,Vietnamese Restaurant,Women's Store,Yoga Studio
3,Bar,9,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
71,Bar,9,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [81]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=50)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, pos, cluster in zip(chicago_top_location_merged['lat'], chicago_top_location_merged['lng'], chicago_top_location_merged['categories'], chicago_top_location_merged['Cluster Labels']):
    label = folium.Popup(str(pos) + ': Lat:'+str(lat)+ ' ,Lng:' + str(lon)+ ' ,Cluster:' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters.save('ClusterMap.html')

